In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import string 
import spacy
from tqdm import tqdm
import os
import random
from spacy.util import compounding
from spacy.util import minibatch


import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tse-spacy-model/models/model_pos/tokenizer
/kaggle/input/tse-spacy-model/models/model_pos/meta.json
/kaggle/input/tse-spacy-model/models/model_pos/vocab/key2row
/kaggle/input/tse-spacy-model/models/model_pos/vocab/vectors
/kaggle/input/tse-spacy-model/models/model_pos/vocab/strings.json
/kaggle/input/tse-spacy-model/models/model_pos/vocab/lexemes.bin
/kaggle/input/tse-spacy-model/models/model_pos/ner/cfg
/kaggle/input/tse-spacy-model/models/model_pos/ner/model
/kaggle/input/tse-spacy-model/models/model_pos/ner/moves
/kaggle/input/tse-spacy-model/models/model_neu/tokenizer
/kaggle/input/tse-spacy-model/models/model_neu/meta.json
/kaggle/input/tse-spacy-model/models/model_neu/vocab/key2row
/kaggle/input/tse-spacy-model/models/model_neu/vocab/vectors
/kaggle/input/tse-spacy-model/models/model_neu/vocab/strings.jso

In [3]:

train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [6]:
train_df['text_cleaned'] = train_df['text'].apply(lambda x:clean_text(x))
train_df['selected_text_cleaned'] = train_df['selected_text'].apply(lambda x:clean_text(x))

In [7]:
train_df

,textID,text,selected_text,sentiment,text_cleaned,selected_text_cleaned
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,id have responded if i were going,id have responded if i were going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad i will miss you here in san diego,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,sons of why couldnt they put them on the rel...,sons of
...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish we could come see u on denver husband l...,d lost
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,ive wondered about rake to the client has ma...,dont force
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good for both of you enjoy the break you...,yay good for both of you
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it,but it was worth it


In [8]:
STOPWORDS = stopwords.words('english')
def remove_stopwords(text):
    return [word for word in text.split() if word not in STOPWORDS]

In [9]:
train_df['text_cleaned'] = train_df['text_cleaned'].apply(lambda x : remove_stopwords(x))
train_df['selected_text_cleaned'] = train_df['selected_text_cleaned'].apply(lambda x : remove_stopwords(x))

In [10]:
train_df

,textID,text,selected_text,sentiment,text_cleaned,selected_text_cleaned
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[id, responded, going]","[id, responded, going]"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[sooo, sad, miss, san, diego]","[sooo, sad]"
2,088c60f138,my boss is bullying me...,bullying me,negative,"[boss, bullying]",[bullying]
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[interview, leave, alone]","[leave, alone]"
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[sons, couldnt, put, releases, already, bought]",[sons]
...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[wish, could, come, see, u, denver, husband, l...",[lost]
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[ive, wondered, rake, client, made, clear, net...","[dont, force]"
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,"[yay, good, enjoy, break, probably, need, hect...","[yay, good]"
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,[worth],[worth]


In [11]:
def get_all_words(df_col):
    all_words_text = []
    for row in df_col:
        for word in row:
            all_words_text.append(word)
    return all_words_text

all_words_text = get_all_words(train_df['text_cleaned'])
all_words_selected_text = get_all_words(train_df['selected_text_cleaned'])

In [12]:
all_words_text

['id',
 'responded',
 'going',
 'sooo',
 'sad',
 'miss',
 'san',
 'diego',
 'boss',
 'bullying',
 'interview',
 'leave',
 'alone',
 'sons',
 'couldnt',
 'put',
 'releases',
 'already',
 'bought',
 'shameless',
 'plugging',
 'best',
 'rangers',
 'forum',
 'earth',
 'feedings',
 'baby',
 'fun',
 'smiles',
 'coos',
 'soooo',
 'high',
 'journey',
 'wow',
 'u',
 'became',
 'cooler',
 'hehe',
 'possible',
 'much',
 'love',
 'hopeful',
 'reckon',
 'chances',
 'minimal',
 'p',
 'im',
 'never',
 'gonna',
 'get',
 'cake',
 'stuff',
 'really',
 'really',
 'like',
 'song',
 'love',
 'story',
 'taylor',
 'swift',
 'sharpie',
 'running',
 'dangerously',
 'low',
 'ink',
 'want',
 'go',
 'music',
 'tonight',
 'lost',
 'voice',
 'test',
 'test',
 'lg',
 'uh',
 'oh',
 'sunburned',
 'sok',
 'trying',
 'plot',
 'alternatives',
 'speak',
 'sigh',
 'ive',
 'sick',
 'past',
 'days',
 'thus',
 'hair',
 'looks',
 'wierd',
 'didnt',
 'hat',
 'would',
 'look',
 'back',
 'home',
 'gonna',
 'miss',
 'every',
 'one

In [13]:
all_words_selected_text

['id',
 'responded',
 'going',
 'sooo',
 'sad',
 'bullying',
 'leave',
 'alone',
 'sons',
 'shameless',
 'plugging',
 'best',
 'rangers',
 'forum',
 'earth',
 'fun',
 'soooo',
 'high',
 'wow',
 'u',
 'became',
 'cooler',
 'much',
 'love',
 'hopeful',
 'reckon',
 'chances',
 'minimal',
 'p',
 'im',
 'never',
 'gonna',
 'get',
 'cake',
 'stuff',
 'like',
 'dangerously',
 'lost',
 'test',
 'test',
 'lg',
 'uh',
 'oh',
 'sunburned',
 'sigh',
 'sick',
 'onna',
 'hes',
 'oh',
 'marly',
 'im',
 'sorry',
 'hope',
 'find',
 'soon',
 'interesting',
 'cleaning',
 'house',
 'family',
 'comming',
 'later',
 'today',
 'gotta',
 'restart',
 'computer',
 'thought',
 'supposed',
 'put',
 'end',
 'constant',
 'rebootiness',
 'see',
 'wat',
 'mean',
 'bout',
 'friidays',
 'called',
 'lose',
 'friday',
 'smh',
 'free',
 'fillin',
 'app',
 'ipod',
 'fun',
 'im',
 'addicted',
 'im',
 'sorry',
 'internet',
 'fun',
 'power',
 'back',
 'working',
 'quiteheavenly',
 'hope',
 'well',
 'much',
 'unhappy',
 'minut

In [14]:
def most_common_words(all_words):
    return nltk.FreqDist(all_words)

In [16]:
# for 'text' column
words = []
frequency = []
freqDist = most_common_words(all_words_text)
for word, freq in freqDist.most_common(20):
        words.append(word)
        frequency.append(freq)

for i in range(20):
    print(f'{words[i]} : {frequency[i]}')



im : 3020
day : 2044
good : 1549
get : 1426
like : 1346
go : 1267
dont : 1200
love : 1122
work : 1112
going : 1096
today : 1096
got : 1072
cant : 1020
happy : 976
one : 971
lol : 948
time : 942
know : 930
u : 923
really : 908


In [18]:
all_words_neutral = get_all_words(train_df[train_df['sentiment'] == 'neutral']['text_cleaned'])
all_words_positive = get_all_words(train_df[train_df['sentiment'] == 'positive']['text_cleaned'])
all_words_negative = get_all_words(train_df[train_df['sentiment'] == 'negative']['text_cleaned'])


In [19]:
all_words_negative

['sooo',
 'sad',
 'miss',
 'san',
 'diego',
 'boss',
 'bullying',
 'interview',
 'leave',
 'alone',
 'sons',
 'couldnt',
 'put',
 'releases',
 'already',
 'bought',
 'sharpie',
 'running',
 'dangerously',
 'low',
 'ink',
 'want',
 'go',
 'music',
 'tonight',
 'lost',
 'voice',
 'uh',
 'oh',
 'sunburned',
 'sok',
 'trying',
 'plot',
 'alternatives',
 'speak',
 'sigh',
 'ive',
 'sick',
 'past',
 'days',
 'thus',
 'hair',
 'looks',
 'wierd',
 'didnt',
 'hat',
 'would',
 'look',
 'back',
 'home',
 'gonna',
 'miss',
 'every',
 'one',
 'im',
 'sorry',
 'way',
 'malaysiano',
 'internet',
 'access',
 'twit',
 'went',
 'sleep',
 'power',
 'cut',
 'noida',
 'power',
 'back',
 'working',
 'consolation',
 'got',
 'bmi',
 'tested',
 'hahaha',
 'says',
 'obesed',
 'well',
 'much',
 'unhappy',
 'minutes',
 'born',
 'raised',
 'nyc',
 'living',
 'texas',
 'past',
 'years',
 'still',
 'miss',
 'ny',
 'im',
 'soooooo',
 'sleeeeepy',
 'last',
 'day',
 'school',
 'todaysniffle',
 'egh',
 'blah',
 'boooooo

In [20]:
words = []
frequency = []
freqDist = most_common_words(all_words_neutral)
for word, freq in freqDist.most_common(20):
        words.append(word)
        frequency.append(freq)

for i in range(20):
    print(f'{words[i]} : {frequency[i]}')

im : 1054
get : 623
go : 576
day : 503
dont : 491
going : 479
work : 477
like : 457
lol : 454
got : 453
today : 435
time : 425
know : 415
back : 410
one : 402
u : 383
see : 353
cant : 344
home : 336
want : 325


In [21]:
words = []
frequency = []
freqDist = most_common_words(all_words_positive)
for word, freq in freqDist.most_common(20):
        words.append(word)
        frequency.append(freq)

for i in range(20):
    print(f'{words[i]} : {frequency[i]}')

day : 1223
good : 1046
love : 875
happy : 840
im : 740
mothers : 627
thanks : 553
great : 478
like : 413
hope : 396
get : 367
fun : 344
today : 342
lol : 338
nice : 334
u : 326
night : 319
really : 307
got : 302
one : 292


In [23]:
# for 'negative' sentiment
words = []
frequency = []
freqDist = most_common_words(all_words_negative)
for word, freq in freqDist.most_common(20):
        words.append(word)
        frequency.append(freq)

for i in range(20):
    print(f'{words[i]} : {frequency[i]}')



im : 1226
like : 476
dont : 469
cant : 463
get : 436
miss : 422
go : 401
sad : 389
work : 386
sorry : 340
really : 339
going : 334
today : 319
day : 318
got : 317
bad : 300
feel : 285
one : 277
hate : 262
know : 262


In [24]:
df_train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
df_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [25]:
df_train['Num_words_text'] = df_train['text'].apply(lambda x:len(str(x).split())) 

In [26]:
df_train

,textID,text,selected_text,sentiment,Num_words_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,7
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,10
2,088c60f138,my boss is bullying me...,bullying me,negative,5
3,9642c003ef,what interview! leave me alone,leave me alone,negative,5
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,14
...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,16
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,23
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,22
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,6


In [27]:
df_train = df_train[df_train['Num_words_text']>=3]


In [28]:
def save_model(output_dir, nlp, new_model_name):
    ''' This Function Saves model to 
    given output directory'''
    
    output_dir = f'../working/{output_dir}'
    if output_dir is not None:        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [29]:

def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()


        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts,  # batch of texts
                            annotations,  # batch of annotations
                            drop=0.5,   # dropout - make it harder to memorise data
                            losses=losses, 
                            )
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [30]:
def get_model_out_path(sentiment):
    '''
    Returns Model output path
    '''
    model_out_path = None
    if sentiment == 'positive':
        model_out_path = 'models/model_pos'
    elif sentiment == 'negative':
        model_out_path = 'models/model_neg'
    return model_out_path


In [31]:
def get_training_data(sentiment):
    '''
    Returns Trainong data in the format needed to train spacy NER
    '''
    train_data = []
    for index, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    return train_data

In [33]:
sentiment = 'positive'

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)
train(train_data, model_path, n_iter=7, model=None)

  0%|          | 0/7 [00:00<?, ?it/s]

Created blank 'en' model


  0%|          | 0/7 [00:39<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
sentiment = 'negative'

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)

train(train_data, model_path, n_iter=7, model=None)

In [ ]:

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)

train(train_data, model_path, n_iter=7, model=None)

In [ ]:
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text
sele

In [ ]:
selected_texts = []
MODELS_BASE_PATH = '../input/tse-spacy-model/models/'

if MODELS_BASE_PATH is not None:
    print("Loading Models  from ", MODELS_BASE_PATH)
    model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
    model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')
        
    for index, row in df_test.iterrows():
        text = row.text
        output_str = ""
        if row.sentiment == 'neutral' or len(text.split()) <= 2:
            selected_texts.append(text)
        elif row.sentiment == 'positive':
            selected_texts.append(predict_entities(text, model_pos))
        else:
            selected_texts.append(predict_entities(text, model_neg))
        
df_test['selected_text'] = selected_texts


In [ ]:
df_submission['selected_text'] = df_test['selected_text']
df_submission.to_csv("submission.csv", index=False)
display(df_submission.head(10))